# Data Analysis

In [ ]:
import math
import IPython

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import preprocessing

pd.options.display.max_columns = None

In [ ]:
raw_train_credit_application = pd.read_csv('data/application_train.csv')
raw_test_credit_application = pd.read_csv('data/application_test.csv')

# raw_bureau = pd.read_csv('data/bureau.csv')
# raw_bureau_balance = pd.read_csv('data/bureau_balance.csv')
# raw_credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
# raw_installments_payments = pd.read_csv('data/installments_payments.csv')
# raw_pos_cash_balance = pd.read_csv('data/pos_cash_balance.csv')
# raw_previous_application = pd.read_csv('data/previous_application.csv')

In [ ]:
%matplotlib inline

plt.figure(figsize=[15, 100])
columns = ['NAME_CONTRACT_TYPE', 'CODE_GENDER',
           'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
           'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
           'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL',
           'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
           'FLAG_PHONE', 'FLAG_EMAIL', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
           'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
           'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
           'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
           'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
           'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
           'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'
          ]
row_no = 20
column_no = 3
for index, column in enumerate(columns):
    plt.subplot(row_no, column_no, index + 1)
    plt.title(column)
    raw_train_credit_application.groupby([column]).TARGET.value_counts().plot(kind='bar')
    
plt.tight_layout()

# Data Cleaning

In [ ]:
# def clean_data(df):
#     new_ids = df['SK_ID_CURR'].to_frame()
#     new_data = pd.DataFrame()
#     if 'TARGET' in df.columns:
#         new_targets = df['TARGET'].to_frame()
#     else:
#         new_targets = None
    
#     # NAME_CONTRACT_TYPE
#     new_data['NAME_CONTRACT_TYPE'] = df['NAME_CONTRACT_TYPE']
    
#     # CODE_GENDER
#     new_data['CODE_GENDER'] = df['CODE_GENDER'].apply(lambda val: 'F' if val == 'XNA' else val)
    
#     # NAME_TYPE_SUITE
#     new_data['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE']
    
#     # CNT_CHILDREN
#     new_data['CNT_CHILDREN'] = df['CNT_CHILDREN'].apply(lambda val: 3 if val > 3 else val)
    
#     # AMT
#     new_data['AMT_CREDIT'] = df['AMT_CREDIT'].apply(lambda val: 1800000 if val > 1800000 else val)
#     new_data['AMT_INCOME_TOTAL'] = df['AMT_INCOME_TOTAL'].apply(lambda val: 500000 if val > 500000 else val)
#     new_data['AMT_ANNUITY'] = raw_train_credit_application['AMT_ANNUITY'].apply(lambda val: 100000 if val > 100000 else val)
#     new_data['AMT_GOODS_PRICE'] = raw_train_credit_application['AMT_GOODS_PRICE'].apply(lambda val: 2500000 if val > 2500000 else val)

#     # FLAGS
#     # new_data['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].apply(lambda val: '0' if val == 'N' else 1)
#     # new_data['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].apply(lambda val: '0' if val == 'N' else 1)
#     # new_data['FLAG_MOBIL'] = df['FLAG_MOBIL']
#     # new_data['FLAG_EMP_PHONE'] = df['FLAG_EMP_PHONE']
#     # new_data['FLAG_WORK_PHONE'] = df['FLAG_WORK_PHONE']
#     # new_data['FLAG_CONT_MOBILE'] = df['FLAG_CONT_MOBILE']
#     # new_data['FLAG_PHONE'] = df['FLAG_PHONE']
#     # new_data['FLAG_EMAIL'] = df['FLAG_EMAIL']
#     # new_data['FLAG_DOCUMENT_2'] = df['FLAG_DOCUMENT_2']
#     # new_data['FLAG_DOCUMENT_3'] = df['FLAG_DOCUMENT_3']
#     # new_data['FLAG_DOCUMENT_4'] = df['FLAG_DOCUMENT_4']
#     # new_data['FLAG_DOCUMENT_5'] = df['FLAG_DOCUMENT_5']
#     # new_data['FLAG_DOCUMENT_6'] = df['FLAG_DOCUMENT_6']
#     # new_data['FLAG_DOCUMENT_7'] = df['FLAG_DOCUMENT_7']
#     # new_data['FLAG_DOCUMENT_8'] = df['FLAG_DOCUMENT_8']
#     # new_data['FLAG_DOCUMENT_9'] = df['FLAG_DOCUMENT_9']
#     # new_data['FLAG_DOCUMENT_10'] = df['FLAG_DOCUMENT_10']
#     # new_data['FLAG_DOCUMENT_11'] = df['FLAG_DOCUMENT_11']
#     # new_data['FLAG_DOCUMENT_12'] = df['FLAG_DOCUMENT_12']
#     # new_data['FLAG_DOCUMENT_13'] = df['FLAG_DOCUMENT_13']
#     # new_data['FLAG_DOCUMENT_14'] = df['FLAG_DOCUMENT_14']
#     # new_data['FLAG_DOCUMENT_15'] = df['FLAG_DOCUMENT_15']
#     # new_data['FLAG_DOCUMENT_16'] = df['FLAG_DOCUMENT_16']
#     # new_data['FLAG_DOCUMENT_17'] = df['FLAG_DOCUMENT_17']
#     # new_data['FLAG_DOCUMENT_18'] = df['FLAG_DOCUMENT_18']
#     # new_data['FLAG_DOCUMENT_19'] = df['FLAG_DOCUMENT_19']
#     # new_data['FLAG_DOCUMENT_20'] = df['FLAG_DOCUMENT_20']
#     # new_data['FLAG_DOCUMENT_21'] = df['FLAG_DOCUMENT_21']
    
#     new_data = pd.get_dummies(new_data, columns=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE'])
    
#     return new_ids, new_data, new_targets

In [ ]:
def clean_data(df, train_df=None):     
    new_df = df.copy()
    
    if train_df is None:
        for col in new_df.columns.values:
            if new_df[col].dtype == np.float or new_df[col].dtype == np.int:
                new_df[col] = new_df[col].fillna(new_df[col].median())
            elif new_df[col].dtype == np.object:
                new_df[col] = new_df[col].fillna(new_df[col].value_counts().idxmax())
    else:
        for col in new_df.columns.values:
            if new_df[col].dtype == np.float64 or new_df[col].dtype == np.int64:
                new_df[col] = new_df[col].fillna(train_df[col].median())
            elif new_df[col].dtype == np.object:
                new_df[col] = new_df[col].fillna(train_df[col].value_counts().idxmax())
    
    new_df['SK_ID_CURR'] = new_df['SK_ID_CURR']
    if 'TARGET' in new_df:
        new_df['TARGET'] = new_df['TARGET']
    new_df['NAME_CONTRACT_TYPE'] = new_df['NAME_CONTRACT_TYPE']
    new_df['CODE_GENDER'] = new_df['CODE_GENDER'].apply(lambda val: 'F' if val == 'XNA' else val)
    new_df['FLAG_OWN_CAR'] = new_df['FLAG_OWN_CAR'].apply(lambda val: '0' if val == 'N' else 1)
    new_df['FLAG_OWN_REALTY'] = new_df['FLAG_OWN_REALTY'].apply(lambda val: '0' if val == 'N' else 1)
    new_df['CNT_CHILDREN'] = new_df['CNT_CHILDREN'].apply(lambda val: 3 if val > 3 else val)
    new_df['AMT_INCOME_TOTAL'] = new_df['AMT_INCOME_TOTAL'].apply(lambda val: 500000 if val > 500000 else val)
    new_df['AMT_CREDIT'] = new_df['AMT_CREDIT'].apply(lambda val: 1800000 if val > 1800000 else val)
    new_df['AMT_ANNUITY'] = new_df['AMT_ANNUITY'].apply(lambda val: 100000 if val > 100000 else val)
    new_df['AMT_GOODS_PRICE'] = new_df['AMT_GOODS_PRICE'].apply(lambda val: 2500000 if val > 2500000 else val)
    new_df['NAME_TYPE_SUITE'] = new_df['NAME_TYPE_SUITE']
    new_df['NAME_INCOME_TYPE'] = new_df['NAME_INCOME_TYPE']
    new_df['NAME_EDUCATION_TYPE'] = new_df['NAME_EDUCATION_TYPE']
    new_df['NAME_FAMILY_STATUS'] = new_df['NAME_FAMILY_STATUS'].apply(lambda val: 'Married' if val == 'Unknown' else val)
    new_df['NAME_HOUSING_TYPE'] = new_df['NAME_HOUSING_TYPE']
    new_df['REGION_POPULATION_RELATIVE'] = new_df['REGION_POPULATION_RELATIVE']
    new_df['DAYS_BIRTH'] = new_df['DAYS_BIRTH']
    new_df['DAYS_EMPLOYED'] = new_df['DAYS_EMPLOYED'].apply(lambda val: 0 if val > 0 else val)
    new_df['DAYS_REGISTRATION'] = new_df['DAYS_REGISTRATION'].apply(lambda val: -18000 if val < -18000 else val)
    new_df['DAYS_ID_PUBLISH'] = new_df['DAYS_ID_PUBLISH'].apply(lambda val: -6300 if val < -6300 else val)
    new_df['OWN_CAR_AGE'] = new_df['OWN_CAR_AGE'].apply(lambda val: 65 if val > 65 else val)
    new_df['FLAG_MOBIL'] = new_df['FLAG_MOBIL']
    new_df['FLAG_EMP_PHONE'] = new_df['FLAG_EMP_PHONE']
    new_df['FLAG_WORK_PHONE'] = new_df['FLAG_WORK_PHONE']
    new_df['FLAG_CONT_MOBILE'] = new_df['FLAG_CONT_MOBILE']
    new_df['FLAG_PHONE'] = new_df['FLAG_PHONE']
    new_df['FLAG_EMAIL'] = new_df['FLAG_EMAIL']
    new_df['OCCUPATION_TYPE'] = new_df['OCCUPATION_TYPE']
    new_df['CNT_FAM_MEMBERS'] = new_df['CNT_FAM_MEMBERS']
    new_df['REGION_RATING_CLIENT'] = new_df['REGION_RATING_CLIENT']
    new_df['REGION_RATING_CLIENT_W_CITY'] = new_df['REGION_RATING_CLIENT_W_CITY']
    new_df['WEEKDAY_APPR_PROCESS_START'] = new_df['WEEKDAY_APPR_PROCESS_START']
    new_df['HOUR_APPR_PROCESS_START'] = new_df['HOUR_APPR_PROCESS_START']
    new_df['REG_REGION_NOT_LIVE_REGION'] = new_df['REG_REGION_NOT_LIVE_REGION']
    new_df['REG_REGION_NOT_WORK_REGION'] = new_df['REG_REGION_NOT_WORK_REGION']
    new_df['LIVE_REGION_NOT_WORK_REGION'] = new_df['LIVE_REGION_NOT_WORK_REGION']
    new_df['REG_CITY_NOT_LIVE_CITY'] = new_df['REG_CITY_NOT_LIVE_CITY']
    new_df['REG_CITY_NOT_WORK_CITY'] = new_df['REG_CITY_NOT_WORK_CITY']
    new_df['LIVE_CITY_NOT_WORK_CITY'] = new_df['LIVE_CITY_NOT_WORK_CITY']
    new_df['ORGANIZATION_TYPE'] = new_df['ORGANIZATION_TYPE'].apply(lambda val: 'Unknown' if val == 'XNA' else val)
    new_df['EXT_SOURCE_1'] = new_df['EXT_SOURCE_1']
    new_df['EXT_SOURCE_2'] = new_df['EXT_SOURCE_2']
    new_df['EXT_SOURCE_3'] = new_df['EXT_SOURCE_3']
    new_df['APARTMENTS_AVG'] = new_df['APARTMENTS_AVG']
    new_df['BASEMENTAREA_AVG'] = new_df['BASEMENTAREA_AVG'].apply(lambda val: 0.5 if val > 0.5 else val)
    new_df['YEARS_BEGINEXPLUATATION_AVG'] = new_df['YEARS_BEGINEXPLUATATION_AVG'].apply(lambda val: 0.9 if val < 0.9 else val)
    new_df['YEARS_BUILD_AVG'] = new_df['YEARS_BUILD_AVG']
    new_df['COMMONAREA_AVG'] = new_df['COMMONAREA_AVG'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['ELEVATORS_AVG'] = new_df['ELEVATORS_AVG'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['ENTRANCES_AVG'] = new_df['ENTRANCES_AVG'].apply(lambda val: 0.5 if val > 0.5 else val)
    new_df['FLOORSMAX_AVG'] = new_df['FLOORSMAX_AVG']
    new_df['FLOORSMIN_AVG'] = new_df['FLOORSMIN_AVG']
    new_df['LANDAREA_AVG'] = new_df['LANDAREA_AVG'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['LIVINGAPARTMENTS_AVG'] = new_df['LIVINGAPARTMENTS_AVG'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['LIVINGAREA_AVG'] = new_df['LIVINGAREA_AVG'].apply(lambda val: 0.75 if val > 0.75 else val)
    new_df['NONLIVINGAPARTMENTS_AVG'] = new_df['NONLIVINGAPARTMENTS_AVG'].apply(lambda val: 0.075 if val > 0.075 else val)
    new_df['NONLIVINGAREA_AVG'] = new_df['NONLIVINGAREA_AVG'].apply(lambda val: 0.3 if val > 0.3 else val)
    new_df['APARTMENTS_MODE'] = new_df['APARTMENTS_MODE'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['BASEMENTAREA_MODE'] = new_df['BASEMENTAREA_MODE'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['YEARS_BEGINEXPLUATATION_MODE'] = new_df['YEARS_BEGINEXPLUATATION_MODE'].apply(lambda val: 0.95 if val < 0.95 else val)
    new_df['YEARS_BUILD_MODE'] = new_df['YEARS_BUILD_MODE'].apply(lambda val: 0.3 if val < 0.3 else val)
    new_df['COMMONAREA_MODE'] = new_df['COMMONAREA_MODE'].apply(lambda val: 0.25 if val > 0.25 else val)
    new_df['ELEVATORS_MODE'] = new_df['ELEVATORS_MODE'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['ENTRANCES_MODE'] = new_df['ENTRANCES_MODE'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['FLOORSMAX_MODE'] = new_df['FLOORSMAX_MODE'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['FLOORSMIN_MODE'] = new_df['FLOORSMIN_MODE'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['LANDAREA_MODE'] = new_df['LANDAREA_MODE'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['LIVINGAPARTMENTS_MODE'] = new_df['LIVINGAPARTMENTS_MODE'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['LIVINGAREA_MODE'] = new_df['LIVINGAREA_MODE'].apply(lambda val: 0.75 if val > 0.75 else val)
    new_df['NONLIVINGAPARTMENTS_MODE'] = new_df['NONLIVINGAPARTMENTS_MODE'].apply(lambda val: 0.075 if val > 0.075 else val)
    new_df['NONLIVINGAREA_MODE'] = new_df['NONLIVINGAREA_MODE'].apply(lambda val: 0.2 if val > 0.2 else val)
    new_df['APARTMENTS_MEDI'] = new_df['APARTMENTS_MEDI'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['BASEMENTAREA_MEDI'] = new_df['BASEMENTAREA_MEDI'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['YEARS_BEGINEXPLUATATION_MEDI'] = new_df['YEARS_BEGINEXPLUATATION_MEDI'].apply(lambda val: 0.95 if val < 0.95 else val)
    new_df['YEARS_BUILD_MEDI'] = new_df['YEARS_BUILD_MEDI'].apply(lambda val: 0.3 if val < 0.3 else val)
    new_df['COMMONAREA_MEDI'] = new_df['COMMONAREA_MEDI'].apply(lambda val: 0.25 if val > 0.25 else val)
    new_df['ELEVATORS_MEDI'] = new_df['ELEVATORS_MEDI'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['ENTRANCES_MEDI'] = new_df['ENTRANCES_MEDI'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['FLOORSMAX_MEDI'] = new_df['FLOORSMAX_MEDI'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['FLOORSMIN_MEDI'] = new_df['FLOORSMIN_MEDI'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['LANDAREA_MEDI'] = new_df['LANDAREA_MEDI'].apply(lambda val: 0.4 if val > 0.4 else val)
    new_df['LIVINGAPARTMENTS_MEDI'] = new_df['LIVINGAPARTMENTS_MEDI'].apply(lambda val: 0.6 if val > 0.6 else val)
    new_df['LIVINGAREA_MEDI'] = new_df['LIVINGAREA_MEDI'].apply(lambda val: 0.75 if val > 0.75 else val)
    new_df['NONLIVINGAPARTMENTS_MEDI'] = new_df['NONLIVINGAPARTMENTS_MEDI'].apply(lambda val: 0.075 if val > 0.075 else val)
    new_df['NONLIVINGAREA_MEDI'] = new_df['NONLIVINGAREA_MEDI'].apply(lambda val: 0.2 if val > 0.2 else val)
    new_df['FONDKAPREMONT_MODE'] = new_df['FONDKAPREMONT_MODE']
    new_df['HOUSETYPE_MODE'] = new_df['HOUSETYPE_MODE']
    new_df['TOTALAREA_MODE'] = new_df['TOTALAREA_MODE'].apply(lambda val: 0.60 if val > 0.60 else val)
    new_df['WALLSMATERIAL_MODE'] = new_df['WALLSMATERIAL_MODE']
    new_df['EMERGENCYSTATE_MODE'] = new_df['EMERGENCYSTATE_MODE']
    new_df['OBS_30_CNT_SOCIAL_CIRCLE'] = new_df['OBS_30_CNT_SOCIAL_CIRCLE'].apply(lambda val: 25 if val > 25 else val)
    new_df['DEF_30_CNT_SOCIAL_CIRCLE'] = new_df['DEF_30_CNT_SOCIAL_CIRCLE'].apply(lambda val: 5 if val > 5 else val)
    new_df['OBS_60_CNT_SOCIAL_CIRCLE'] = new_df['OBS_60_CNT_SOCIAL_CIRCLE'].apply(lambda val: 15 if val > 15 else val)
    new_df['DEF_60_CNT_SOCIAL_CIRCLE'] = new_df['DEF_60_CNT_SOCIAL_CIRCLE'].apply(lambda val: 3 if val > 3 else val)
    new_df['DAYS_LAST_PHONE_CHANGE'] = new_df['DAYS_LAST_PHONE_CHANGE'].apply(lambda val: -3200 if val < -3200 else val)
    new_df['FLAG_DOCUMENT_2'] = new_df['FLAG_DOCUMENT_2']
    new_df['FLAG_DOCUMENT_3'] = new_df['FLAG_DOCUMENT_3']
    new_df['FLAG_DOCUMENT_4'] = new_df['FLAG_DOCUMENT_4']
    new_df['FLAG_DOCUMENT_5'] = new_df['FLAG_DOCUMENT_5']
    new_df['FLAG_DOCUMENT_6'] = new_df['FLAG_DOCUMENT_6']
    new_df['FLAG_DOCUMENT_7'] = new_df['FLAG_DOCUMENT_7']
    new_df['FLAG_DOCUMENT_8'] = new_df['FLAG_DOCUMENT_8']
    new_df['FLAG_DOCUMENT_9'] = new_df['FLAG_DOCUMENT_9']
    new_df['FLAG_DOCUMENT_10'] = new_df['FLAG_DOCUMENT_10']
    new_df['FLAG_DOCUMENT_11'] = new_df['FLAG_DOCUMENT_11']
    new_df['FLAG_DOCUMENT_12'] = new_df['FLAG_DOCUMENT_12']
    new_df['FLAG_DOCUMENT_13'] = new_df['FLAG_DOCUMENT_13']
    new_df['FLAG_DOCUMENT_14'] = new_df['FLAG_DOCUMENT_14']
    new_df['FLAG_DOCUMENT_15'] = new_df['FLAG_DOCUMENT_15']
    new_df['FLAG_DOCUMENT_16'] = new_df['FLAG_DOCUMENT_16']
    new_df['FLAG_DOCUMENT_17'] = new_df['FLAG_DOCUMENT_17']
    new_df['FLAG_DOCUMENT_18'] = new_df['FLAG_DOCUMENT_18']
    new_df['FLAG_DOCUMENT_19'] = new_df['FLAG_DOCUMENT_19']
    new_df['FLAG_DOCUMENT_20'] = new_df['FLAG_DOCUMENT_20']
    new_df['FLAG_DOCUMENT_21'] = new_df['FLAG_DOCUMENT_21']
    new_df['AMT_REQ_CREDIT_BUREAU_HOUR'] = new_df['AMT_REQ_CREDIT_BUREAU_HOUR'].apply(lambda val: 2.0 if val > 2.0 else val)
    new_df['AMT_REQ_CREDIT_BUREAU_DAY'] = new_df['AMT_REQ_CREDIT_BUREAU_DAY'].apply(lambda val: 4.0 if val > 4.0 else val)
    new_df['AMT_REQ_CREDIT_BUREAU_WEEK'] = new_df['AMT_REQ_CREDIT_BUREAU_WEEK'].apply(lambda val: 3.0 if val > 3.0 else val)
    new_df['AMT_REQ_CREDIT_BUREAU_MON'] = new_df['AMT_REQ_CREDIT_BUREAU_MON'].apply(lambda val: 17.0 if val > 17.0 else val)
    new_df['AMT_REQ_CREDIT_BUREAU_QRT'] = new_df['AMT_REQ_CREDIT_BUREAU_QRT'].apply(lambda val: 6.0 if val > 6.0 else val)
    new_df['AMT_REQ_CREDIT_BUREAU_YEAR'] = new_df['AMT_REQ_CREDIT_BUREAU_YEAR'].apply(lambda val: 14.0 if val > 14.0 else val)
    
    return new_df

In [ ]:
train_credit_application = clean_data(df=raw_train_credit_application)
test_credit_application = clean_data(df=raw_test_credit_application, train_df=raw_train_credit_application)

# Feature Selection

# Feature Engineering

In [ ]:
def engineer_feature(df):
    columns = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 
               'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 
               'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 
               'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 
               'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 
               'EMERGENCYSTATE_MODE']
    for col in columns:
        if col in df:
            df = pd.get_dummies(df, columns=[col])
    
    return df

In [ ]:
train_credit_application = engineer_feature(train_credit_application)
test_credit_application = engineer_feature(test_credit_application)

# Data Splitting

In [ ]:
train_credit_application = train_credit_application.reindex(np.random.permutation(train_credit_application.index))

Check distribution if the randomization is fine

In [ ]:
total_count = train_credit_application['SK_ID_CURR'].count()
training_count = math.ceil(total_count * 0.75)
validation_count = math.floor(total_count * 0.25)

In [ ]:
training_credit_application = train_credit_application.head(training_count)
validation_credit_application = train_credit_application.tail(validation_count)
testing_credit_application = test_credit_application

In [ ]:
def split_ids_data_targets(df, dataframe=False):
    new_df = df.copy()
    
    new_df_ids = new_df['SK_ID_CURR']
    if 'TARGET' in new_df:
        new_df_targets = new_df['TARGET']
        new_df_data = new_df.drop(columns=['SK_ID_CURR', 'TARGET'])
    else:
        new_df_targets = None
        new_df_data = new_df.drop(columns=['SK_ID_CURR'])
    
    
    if not dataframe:
        new_df_ids = new_df_ids.values
        new_df_data = new_df_data.values
        if new_df_targets is not None:
            new_df_targets = new_df_targets.values
    
    return new_df_ids, new_df_data, new_df_targets

In [ ]:
training_ids, training_data, training_targets = split_ids_data_targets(training_credit_application)
validation_ids, validation_data, validation_targets = split_ids_data_targets(validation_credit_application)
testing_ids, testing_data, testing_targets = split_ids_data_targets(testing_credit_application)

training_targets_onehot = (preprocessing.OneHotEncoder().fit_transform(training_targets.reshape(-1, 1))).toarray()
validation_targets_onehot = (preprocessing.OneHotEncoder().fit_transform(validation_targets.reshape(-1, 1))).toarray()

# Data Model

In [ ]:
import tensorflow as tf

from sklearn import metrics
from sklearn import linear_model, ensemble, svm

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers

sgd_classifier =  linear_model.SGDClassifier()
sgd_classifier.fit(training_data, training_targets)

svm_classifier = svm.SVC(kernel='rbf')
svm_classifier.fit(training_data, training_targets)

In [ ]:
def auc_metric(y_true, y_pred):
    return tf.Variable(metrics.roc_auc_score(y_true, y_pred), name='auc_score')

In [ ]:
default_activation = 'sigmoid'
default_last_activation = 'sigmoid'
default_batch_size = 1000
default_epochs = 10

default_dnn_classifier_settings = [
    # Adam
    {'optimizer': optimizers.Adam(), 'batch_size': default_batch_size, 'epochs': default_epochs, 
     'activation': default_activation, 'last_activation': default_last_activation,
     'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
    
    # SGD
#     {'optimizer': optimizers.SGD(momentum=0.1), 'batch_size': default_batch_size, 'epochs': default_epochs * 10,
#      'activation': default_activation, 'last_activation': default_last_activation,
#      'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
    
    # Adagrad
    {'optimizer': optimizers.Adagrad(), 'batch_size': default_batch_size, 'epochs': default_epochs, 
     'activation': default_activation, 'last_activation': default_last_activation,
     'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
    
    # RMSprop
    {'optimizer': optimizers.RMSprop(), 'batch_size': default_batch_size, 'epochs': default_epochs, 
     'activation': default_activation, 'last_activation': default_last_activation,
     'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
    
    # Adamax
    {'optimizer': optimizers.Adamax(), 'batch_size': default_batch_size, 'epochs': default_epochs, 
     'activation': default_activation, 'last_activation': default_last_activation,
     'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
    
    # Nadam
    {'optimizer': optimizers.Nadam(), 'batch_size': default_batch_size, 'epochs': default_epochs, 
     'activation': default_activation, 'last_activation': default_last_activation,
     'acc': None, 'val_acc': None, 'history': None, 'auc': None, 'val_auc': None},
]

dnn_classifier_settings = []
dnn_classifier_settings.extend(default_dnn_classifier_settings)

new_settings = [
]

if len(new_settings) > 0:
    dnn_classifier_settings.extend(new_settings)

In [ ]:
for index, dnn_classifier_setting in enumerate(dnn_classifier_settings):
    if dnn_classifier_setting['acc'] is None and dnn_classifier_setting['val_acc'] is None and dnn_classifier_setting['history'] is None:
        optimizer = dnn_classifier_setting['optimizer']
        batch_size = dnn_classifier_setting['batch_size']
        epochs = dnn_classifier_setting['epochs']
        activation = dnn_classifier_setting['activation']
        last_activation = dnn_classifier_setting['last_activation']

        dnn_classifier = Sequential()
        input_shape = (training_data.shape[1], )
        dnn_classifier.add(Dense(128, activation=activation, input_shape=input_shape))
        dnn_classifier.add(Dropout(rate=0.35))
        dnn_classifier.add(Dense(128, activation=activation))
        dnn_classifier.add(Dropout(rate=0.35))
        dnn_classifier.add(Dense(64, activation=activation))
        dnn_classifier.add(Dropout(rate=0.25))
        dnn_classifier.add(Dense(2, activation=last_activation))
        dnn_classifier.compile(loss='binary_crossentropy', 
                               optimizer=optimizer,
                               metrics=['acc'])
        history = dnn_classifier.fit(training_data, training_targets_onehot,
                          epochs=epochs, batch_size=batch_size, verbose=0, shuffle=True,
                          validation_data=(validation_data, validation_targets_onehot))

        dnn_classifier_setting['history'] = history
        
        training_predictions = dnn_classifier.predict(training_data)
        training_predictions = pd.DataFrame(training_predictions).apply(lambda val: 1.0 if val[1] > 0.50 else 0.0, axis=1)
        
        validation_predictions = dnn_classifier.predict(validation_data)
        validation_predictions = pd.DataFrame(validation_predictions).apply(lambda val: 1.0 if val[1] > 0.50 else 0.0, axis=1)
        
        dnn_classifier_setting['acc'] = metrics.accuracy_score(training_targets, training_predictions)
        dnn_classifier_setting['val_acc'] = metrics.accuracy_score(validation_targets, validation_predictions)
        dnn_classifier_setting['auc'] = metrics.roc_auc_score(training_targets, training_predictions)
        dnn_classifier_setting['val_auc'] = metrics.roc_auc_score(validation_targets, validation_predictions)

    print('%2d: Optimizer: %10s; LR: %.5f; bs: %3d; epochs: %4d; acc: %.4f; val_acc: %.4f; auc: %.2f; val_auc: %.2f' % (index, 
                                                                                              type(dnn_classifier_setting['optimizer']).__name__, 
                                                                                              dnn_classifier_setting['optimizer'].get_config()['lr'], 
                                                                                              dnn_classifier_setting['batch_size'], 
                                                                                              dnn_classifier_setting['epochs'],
                                                                                              dnn_classifier_setting['acc'], 
                                                                                              dnn_classifier_setting['val_acc'],
                                                                                              dnn_classifier_setting['auc'],
                                                                                              dnn_classifier_setting['val_auc']))

IPython.display.Audio('http://www.pacdv.com/sounds/interface_sound_effects/sound94.wav', autoplay=True)

# Model Evaluation

In [ ]:
classifier = dnn_classifier

In [ ]:
training_predictions = classifier.predict(training_data)
print(training_predictions)
training_predictions = pd.DataFrame(training_predictions).apply(lambda val: 1.0 if val[1] > 0.1 else 0.0, axis=1)
validation_predictions = classifier.predict(validation_data)
validation_predictions = pd.DataFrame(validation_predictions).apply(lambda val: 1.0 if val[1] > 0.1 else 0.0, axis=1)

In [ ]:
training_targets

In [ ]:
pd.Series(training_targets).value_counts()

In [ ]:
training_predictions.value_counts()

In [ ]:
print(metrics.accuracy_score(training_targets, training_predictions))
print(metrics.roc_auc_score(training_targets, training_predictions))
print(metrics.accuracy_score(validation_targets, validation_predictions))
print(metrics.roc_auc_score(validation_targets, validation_predictions))

# Prepare submission.csv